In [1]:
import re, sys, os 
import argparse
from Bio import Align
import pandas as pd


In [2]:
# Part 1 Prepare data
# 1.1
def extract_pdb_name_and_generate_df(filepath):
    # Read the table from the file, assuming comma separation
    # Skip the first two rows which contain 'REM' and 'All-atoms'
    df = pd.read_csv(filepath, delimiter=',', skiprows=1)
    # print(f'df generated from file path: {filepath} \n{df}')
    # Extract the PDB name from the file name
    basename = os.path.basename(filepath)
    pdb_name = basename.split('_')[0]
    rest_name = basename.split('_')[1]
    chain_id = rest_name.split('-')[0]
    
    # Create a new DataFrame that only retains information from column index 1, 3, and 4
    # which are the aa, index, ABS
    selected_indices = [1, 3, 4]
    data_df = df.iloc[: ,selected_indices]
    # Swap the aa and the index columns
    cols = list(data_df.columns)
    cols[0], cols[1] = cols[1], cols[0]  # Swap the column names
    data_df = data_df[cols]  # Reindex the DataFrame with the new column order
    # Assign new column names to the DataFrame
    data_df.columns = ['NUM', 'AA', 'ABS']
    data_df = data_df[['NUM', 'AA', 'ABS']]
    
    return basename, pdb_name, chain_id, data_df

filepath = '/Users/luna/Documents/RP1/2_do_cal_and_result/5w3f_A-AB.csv'
basename, pdb_name, chain_id, data_df = extract_pdb_name_and_generate_df(filepath)
print(f'Loaded relevant information of {pdb_name}, chain {chain_id}, from {basename}')
print(data_df)

Loaded relevant information of 5w3f, chain A, from 5w3f_A-AB.csv
     NUM   AA  ABS
0      1  MET  0.0
1      2  ARG  0.0
2      3  GLU  0.0
3      4  VAL  0.0
4      5  ILE  0.0
..   ...  ...  ...
435  436  VAL  0.0
436  437  GLY  0.0
437  438  ALA  0.0
438  439  ASP  0.0
439  440  SER  0.0

[440 rows x 3 columns]


In [3]:
# 1.2
def convert_three_letter_code_to_one_letter_code(three_letter_list):
    aa_dict = {
        'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D',
        'CYS': 'C', 'GLU': 'E', 'GLN': 'Q', 'GLY': 'G',
        'HIS': 'H', 'ILE': 'I', 'LEU': 'L', 'LYS': 'K',
        'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S',
        'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V',
        # Additional potential codes
        'SEC': 'U', 'PYL': 'O', 'ASX': 'B', 'GLX': 'Z',
        'XLE': 'J', 'XAA': 'X', 'TER': '*', 'UNK': 'X'}
    one_letter_list = []
    for aa in three_letter_list:
        one_letter_code = aa_dict.get(aa, 'X')  # Default to 'X' if aa is not found
        one_letter_list.append(one_letter_code)
        
    return one_letter_list

In [4]:
# 1.3
def process_df_and_generate_sequence_string(data_df):
    # Convert the second column (amino acids) to a list of three-letter codes
    three_letter_list = data_df.iloc[:, 1].tolist()
    # Convert the three-letter codes to one-letter codes using the function
    one_letter_list = convert_three_letter_code_to_one_letter_code(three_letter_list)
    # Replace the second column with the one-letter codes
    data_df.iloc[:, 1] = one_letter_list
    
    # Generate the PDB sequence 
    pdb_sequence = ''.join(one_letter_list)
    
    return pdb_sequence, data_df

pdb_sequence, data_df = process_df_and_generate_sequence_string(data_df)
print(data_df)
print(f'The PDB sequence of {pdb_name} is loaded, length: {len(pdb_sequence)}')
print(pdb_sequence)

     NUM AA  ABS
0      1  M  0.0
1      2  R  0.0
2      3  E  0.0
3      4  V  0.0
4      5  I  0.0
..   ... ..  ...
435  436  V  0.0
436  437  G  0.0
437  438  A  0.0
438  439  D  0.0
439  440  S  0.0

[440 rows x 3 columns]
The PDB sequence of 5w3f is loaded, length: 440
MREVISINVGQAGCQIGNACWELYSLEHGIKPDGHLEDGLSKPKGGEEGFSTFFHETGYGKFVPRAIYVDLEPNVIDEVRNGPYKDLFHPEQLISGKEDAANNYARGHYTVGREILGDVLDRIRKLADQCDGLQGFLFTHSLGGGTGSGLGSLLLEELSAEYGKKSKLEFAVYPAPQVSTSVVEPYNTVLTTHTTLEHADCTFMVDNEAIYDMCKRNLDIPRPSFANLNNLIAQVVSSVTASLRFDGSLNVDLNEFQTNLVPYPRIHFPLVSYSPVLSKSKAFHESNSVSEITNACFEPGNQMVKCDPRDGKYMATCLLYRGDVVTRDVQRAVEQVKNKKTVQLVDWCPTGFKIGICYEPPTATPNSQLATVDRAVCMLSNTTSIAEAWKRIDRKFDLMYAKRAFVHWYVGEGMEEGEFTEAREDLAALERDYIEVGADS


In [5]:
# 1.4
def identify_aa_blocks(pdb_name, data_df):
    # Find the smallest and largest NUM values
    min_num = data_df['NUM'].min()
    max_num = data_df['NUM'].max()
    
    # Generate the full range of numbers from min_num to max_num
    full_range = set(range(int(min_num), int(max_num) + 1))
    print(f'For the corresponding sequence in PDB model {pdb_name}, the PDB aa range: \nmin: {min_num}, max: {max_num}')
    
    # Get the set of NUM values that actually appear in the DataFrame
    actual_nums = set(data_df['NUM'])
    
    # Find the set of numbers that are missing from the DataFrame
    missing_nums = sorted(full_range - actual_nums)
    
    # Identify the existing PDB aa blocks
    print('\nIdentified PDB aa block:')
    current_block_start = None
    for num in sorted(actual_nums):
        if current_block_start is None:
            current_block_start = num
        # If the next number is missing or it's the last number, end the current block
        if num + 1 not in actual_nums or num == max_num:
            # Get the AA for the start and end of the block
            start_aa = data_df.loc[data_df['NUM'] == current_block_start, 'AA'].values[0]
            end_aa = data_df.loc[data_df['NUM'] == num, 'AA'].values[0]
            print(f'{current_block_start}({start_aa})-{num}({end_aa})')
            current_block_start = None
    
    # Print missing PDB aa
    print('\nMissing PDB aa:')
    if not missing_nums:
        print('None')
    else:
        missing_block_start = None
        for i, num in enumerate(missing_nums):
            if missing_block_start is None:
                missing_block_start = num
            # If the next number is not consecutive or it's the last missing number, end the current block
            if i + 1 == len(missing_nums) or missing_nums[i + 1] != num + 1:
                if missing_block_start == num:
                    print(missing_block_start)
                else:
                    print(f'{missing_block_start}-{num}')
                missing_block_start = None

identify_aa_blocks(pdb_name, data_df)

For the corresponding sequence in PDB model 5w3f, the PDB aa range: 
min: 1, max: 440

Identified PDB aa block:
1(M)-440(S)

Missing PDB aa:
None


In [6]:
# 1.5
def extract_uniprot_data(uniprot_filepath):
    # Initialize an empty string to hold the sequence
    uniprot_sequence = ''
    # Initialize a variable to hold the UniProt name
    uniprot_name = ''
    
    # Compile the regular expressions for the header and sequence lines
    header_pattern = re.compile(r'^>sp\|(\w+)\|')
    sequence_pattern = re.compile(r'^[A-Z]+$')
    
    # Open the file for reading
    with open(uniprot_filepath, 'r') as file:
        # Iterate over each line in the file
        for line in file:
            # Check if the line is a header (starts with '>')
            if line.startswith('>'):
                # Use the regular expression to extract the UniProt name
                header_match = header_pattern.match(line)
                if header_match:
                    uniprot_name = header_match.group(1)
                else:
                    print('Invalid header format:', line)
                    return None, None
            else:
                # Check if the line contains only uppercase letters (valid sequence line)
                if sequence_pattern.match(line.strip()):
                    # Remove any whitespace and concatenate to the sequence
                    uniprot_sequence += line.strip()
                else:
                    print('Invalid sequence format:', line)
                    return None, None
    
    return uniprot_name, uniprot_sequence

uniprot_filepath = '/Users/luna/Documents/RP1/2_do_cal_and_result/uniprot_seq.txt'
uniprot_name, uniprot_sequence = extract_uniprot_data(uniprot_filepath)
print(f'The Uniprot sequence of {uniprot_name} is loaded, length: {len(uniprot_sequence)}')
print(uniprot_sequence)


The Uniprot sequence of P09733 is loaded, length: 447
MREVISINVGQAGCQIGNACWELYSLEHGIKPDGHLEDGLSKPKGGEEGFSTFFHETGYGKFVPRAIYVDLEPNVIDEVRNGPYKDLFHPEQLISGKEDAANNYARGHYTVGREILGDVLDRIRKLADQCDGLQGFLFTHSLGGGTGSGLGSLLLEELSAEYGKKSKLEFAVYPAPQVSTSVVEPYNTVLTTHTTLEHADCTFMVDNEAIYDMCKRNLDIPRPSFANLNNLIAQVVSSVTASLRFDGSLNVDLNEFQTNLVPYPRIHFPLVSYSPVLSKSKAFHESNSVSEITNACFEPGNQMVKCDPRDGKYMATCLLYRGDVVTRDVQRAVEQVKNKKTVQLVDWCPTGFKIGICYEPPTATPNSQLATVDRAVCMLSNTTSIAEAWKRIDRKFDLMYAKRAFVHWYVGEGMEEGEFTEAREDLAALERDYIEVGADSYAEEEEF


In [7]:
# Part 2 Perform Alignment
# 2.1
def perform_global_alignment(uniprot_sequence, pdb_sequence):
    # Initialize the aligner
    aligner = Align.PairwiseAligner()
    aligner.mode = 'global'  # Perform global alignment
    aligner.match_score = 1  # Score for identical characters
    aligner.mismatch_score = 0  # Score for non-identical characters
    aligner.open_gap_score = 0  # Score to open a gap
    aligner.extend_gap_score = 0  # Score to extend a gap# Define two protein sequences to be aligned
    # Set sequence
    target = uniprot_sequence
    query = pdb_sequence
    # Perform the alignment
    alignments = aligner.align(target, query)
    # Get the best alignment (usually the first one)
    best_alignment = alignments[0]
    
    return best_alignment

alignment = perform_global_alignment(uniprot_sequence, pdb_sequence)
print(f'Alignment: \n\n{alignment}')

Alignment: 

target            0 MREVISINVGQAGCQIGNACWELYSLEHGIKPDGHLEDGLSKPKGGEEGFSTFFHETGYG
                  0 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
query             0 MREVISINVGQAGCQIGNACWELYSLEHGIKPDGHLEDGLSKPKGGEEGFSTFFHETGYG

target           60 KFVPRAIYVDLEPNVIDEVRNGPYKDLFHPEQLISGKEDAANNYARGHYTVGREILGDVL
                 60 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
query            60 KFVPRAIYVDLEPNVIDEVRNGPYKDLFHPEQLISGKEDAANNYARGHYTVGREILGDVL

target          120 DRIRKLADQCDGLQGFLFTHSLGGGTGSGLGSLLLEELSAEYGKKSKLEFAVYPAPQVST
                120 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
query           120 DRIRKLADQCDGLQGFLFTHSLGGGTGSGLGSLLLEELSAEYGKKSKLEFAVYPAPQVST

target          180 SVVEPYNTVLTTHTTLEHADCTFMVDNEAIYDMCKRNLDIPRPSFANLNNLIAQVVSSVT
                180 ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
query           180 SVVEPYNTVLTTHTTLEHADCTFMVDNEAIYDMCKRNLDIPRPSFANLNNLIAQVVSSVT

target     

In [8]:
# 2.2
def extract_aligned_sequences_from_alignment(alignment):
    # Convert the alignment object to a string
    alignment_str = str(alignment)
    
    # Split the alignment into lines
    lines = alignment_str.strip().split("\n")
    
    # Initialize variables to hold the aligned sequences
    aligned_uniprot = ''
    aligned_pdb = ''

    # Process the alignment block by block
    for i in range(0, len(lines), 4):   # Each block has 4 lines
        # Extract parts of the target and query sequences
        # by removes any leading or trailing whitespace with strip()
        # and then splits the line into parts based on whitespace with split()
        target_line_parts = lines[i].strip().split() 
        query_line_parts = lines[i+2].strip().split()
        
        # Check if the line starts with 'target' or 'query' and has at least 3 parts
        if target_line_parts[0].startswith('target') and len(target_line_parts) > 2:
            aligned_uniprot += target_line_parts[2]  # The sequence part is the third element
        if query_line_parts[0].startswith('query') and len(query_line_parts) > 2:
            aligned_pdb += query_line_parts[2]  # The sequence part is the third element
    
    return aligned_uniprot, aligned_pdb

aligned_uniprot, aligned_pdb = extract_aligned_sequences_from_alignment(alignment)
print(f'aligned_uniprot\n{aligned_uniprot}')
print(f'aligned_pdb\n{aligned_pdb}')

aligned_uniprot
MREVISINVGQAGCQIGNACWELYSLEHGIKPDGHLEDGLSKPKGGEEGFSTFFHETGYGKFVPRAIYVDLEPNVIDEVRNGPYKDLFHPEQLISGKEDAANNYARGHYTVGREILGDVLDRIRKLADQCDGLQGFLFTHSLGGGTGSGLGSLLLEELSAEYGKKSKLEFAVYPAPQVSTSVVEPYNTVLTTHTTLEHADCTFMVDNEAIYDMCKRNLDIPRPSFANLNNLIAQVVSSVTASLRFDGSLNVDLNEFQTNLVPYPRIHFPLVSYSPVLSKSKAFHESNSVSEITNACFEPGNQMVKCDPRDGKYMATCLLYRGDVVTRDVQRAVEQVKNKKTVQLVDWCPTGFKIGICYEPPTATPNSQLATVDRAVCMLSNTTSIAEAWKRIDRKFDLMYAKRAFVHWYVGEGMEEGEFTEAREDLAALERDYIEVGADSYAEEEEF
aligned_pdb
MREVISINVGQAGCQIGNACWELYSLEHGIKPDGHLEDGLSKPKGGEEGFSTFFHETGYGKFVPRAIYVDLEPNVIDEVRNGPYKDLFHPEQLISGKEDAANNYARGHYTVGREILGDVLDRIRKLADQCDGLQGFLFTHSLGGGTGSGLGSLLLEELSAEYGKKSKLEFAVYPAPQVSTSVVEPYNTVLTTHTTLEHADCTFMVDNEAIYDMCKRNLDIPRPSFANLNNLIAQVVSSVTASLRFDGSLNVDLNEFQTNLVPYPRIHFPLVSYSPVLSKSKAFHESNSVSEITNACFEPGNQMVKCDPRDGKYMATCLLYRGDVVTRDVQRAVEQVKNKKTVQLVDWCPTGFKIGICYEPPTATPNSQLATVDRAVCMLSNTTSIAEAWKRIDRKFDLMYAKRAFVHWYVGEGMEEGEFTEAREDLAALERDYIEVGADS-------


In [9]:
# Part 3 Generate dfs and dictionaries from previous data
# 3.1
def create_df_from_aligned_target(aligned_uniprot):
    # Initialize a list to store the data for each row
    data_list = []
    uniprot_index = 1  # Initialize the UniProt index counter
    
    # Iterate over each character in the aligned target sequence
    for index, aa in enumerate(aligned_uniprot, start=1):
        # Check if the character is an amino acid (not a gap '-')
        if aa != '-':
            # Add the index, UniProt index, and amino acid to the list
            data_list.append({'Index': index, 'UniProt_Index': uniprot_index, 'UniProt_AA': aa})
            uniprot_index += 1  # Increment the UniProt index counter
        else:
            # If the character is a gap, add it to the list with an empty UniProt index
            data_list.append({'Index': index, 'UniProt_Index': ' ', 'UniProt_AA': aa})
    
    # Create a DataFrame from the list of data
    uniprot_df = pd.DataFrame(data_list)
    
    return uniprot_df

uniprot_df = create_df_from_aligned_target(aligned_uniprot)
print(uniprot_df)

     Index  UniProt_Index UniProt_AA
0        1              1          M
1        2              2          R
2        3              3          E
3        4              4          V
4        5              5          I
..     ...            ...        ...
442    443            443          E
443    444            444          E
444    445            445          E
445    446            446          E
446    447            447          F

[447 rows x 3 columns]


In [10]:
# # 3.2 
# # For substitution and Addition
# # (1)
# def create_df_from_aligned_query(data_df, aligned_pdb):
#     # Initialize a list to store the index and amino acid data
#     aa_data = []
#     index = data_df['NUM'].iloc[0]  # Initialize the index counter
    
#     # Iterate over each character in the aligned query sequence
#     for aa in aligned_pdb:
#         # Check if the character is an amino acid (not a gap '-')
#         if aa != '-':
#             # Add the index and amino acid to the list
#             aa_data.append({'PDB_Index': index, 'PDB_AA': aa})
#             index += 1  # Increment the index counter
#         else:
#             # If the character is a gap, add it to the list with an empty index
#             aa_data.append({'PDB_Index': ' ', 'PDB_AA': aa})
    
#     # Create a DataFrame from the list of data
#     pdb_df = pd.DataFrame(aa_data)
    
#     return pdb_df

# pdb_df = create_df_from_aligned_query(data_df, aligned_pdb)
# print(pdb_df)

In [ ]:
# (2)
# def create_alignment_df_and_dict(uniprot_df, pdb_df):
#     # Merge the two DataFrames on their index
#     alignment_df = pd.merge(uniprot_df, pdb_df, left_index=True, right_index=True, how='inner')
    
#     # Initialize an empty dictionary to store the data
#     alignment_dict = {}
#     # Iterate over each row in the combined DataFrame
#     for index, row in alignment_df.iterrows():
#         # Create a key-value pair with the desired columns
#         key = (row['Index'], row['UniProt_AA'])
#         value = (row['PDB_Index'], row['PDB_AA'])
        
#         # Add the key-value pair to the dictionary
#         alignment_dict[key] = value
    
#     return alignment_df, alignment_dict

# alignment_df, alignment_dict = create_alignment_df_and_dict(uniprot_df, pdb_df)
# print(f'alignment_df \n{alignment_df}')

# print("Contents of alignment_dict:")
# for key, value in alignment_dict.items():
#     print(key, value)


In [10]:
# 3.2
# For deletion
def create_alignment_df_and_dict(uniprot_df, aligned_pdb):
    # Initialize the 'PDB_Index' column with empty strings
    uniprot_df['PDB_Index'] = ''
    # Add the 'PDB_AA' column to uniprot_df with the characters from aligned_pdb
    uniprot_df['PDB_AA'] = list(aligned_pdb)
    # Create the alignment DataFrame with the correct column order
    alignment_df =uniprot_df
    
    # Iterate over the DataFrame and update 'PDB_Index' based on the condition
    for i, aa in enumerate(uniprot_df['PDB_AA']):
        if aa != '-':
            uniprot_df.at[i, 'PDB_Index'] = uniprot_df.at[i, 'UniProt_Index']
    
    alignment_df = alignment_df[['Index', 'UniProt_Index', 'UniProt_AA', 'PDB_Index', 'PDB_AA']]
    
    # Initialize an empty dictionary to store the data
    alignment_dict = {}
    # Iterate over each row in the combined DataFrame
    for index, row in alignment_df.iterrows():
        # Create a key-value pair with the desired columns
        key = (row['Index'], row['UniProt_AA'])
        value = (row['PDB_Index'], row['PDB_AA'])
        
        # Add the key-value pair to the dictionary
        alignment_dict[key] = value
    
    return alignment_df, alignment_dict

# Deletion
print('\nDeletion')
alignment_df, alignment_dict = create_alignment_df_and_dict(uniprot_df, aligned_pdb)
print('alignment_df')
print(alignment_df)

print('\nalignment_dict')
for key, value in alignment_dict.items():
    print(key, value)


Deletion
alignment_df
     Index  UniProt_Index UniProt_AA PDB_Index PDB_AA
0        1              1          M         1      M
1        2              2          R         2      R
2        3              3          E         3      E
3        4              4          V         4      V
4        5              5          I         5      I
..     ...            ...        ...       ...    ...
442    443            443          E                -
443    444            444          E                -
444    445            445          E                -
445    446            446          E                -
446    447            447          F                -

[447 rows x 5 columns]

alignment_dict
(1, 'M') (1, 'M')
(2, 'R') (2, 'R')
(3, 'E') (3, 'E')
(4, 'V') (4, 'V')
(5, 'I') (5, 'I')
(6, 'S') (6, 'S')
(7, 'I') (7, 'I')
(8, 'N') (8, 'N')
(9, 'V') (9, 'V')
(10, 'G') (10, 'G')
(11, 'Q') (11, 'Q')
(12, 'A') (12, 'A')
(13, 'G') (13, 'G')
(14, 'C') (14, 'C')
(15, 'Q') (15, 'Q')
(16, 'I

In [11]:
# 3.3
def create_aa_rel_dict(data_df):
    # Initialize an empty dictionary
    aa_rel_dict = {}
    # Iterate over each row in the DataFrame
    for pdb_index, row in data_df.iterrows():
        # Add the key-value pair to the dictionary
        aa_rel_dict[(row['NUM'], row['AA'])] = row['ABS']
        
    return aa_rel_dict

aa_rel_dict = create_aa_rel_dict(data_df)
# Debugging: Print the contents of aa_rel_dict
print("Contents of mapping_dict:")
for key, value in aa_rel_dict.items():
    print(key, value)

Contents of mapping_dict:
(1, 'M') 0.0
(2, 'R') 0.0
(3, 'E') 0.0
(4, 'V') 0.0
(5, 'I') 0.0
(6, 'S') 0.0
(7, 'I') 0.0
(8, 'N') 0.0
(9, 'V') 0.0
(10, 'G') 0.0
(11, 'Q') 59.35
(12, 'A') 0.0
(13, 'G') 0.0
(14, 'C') 0.0
(15, 'Q') 3.92
(16, 'I') 0.0
(17, 'G') 0.0
(18, 'N') 0.0
(19, 'A') 0.0
(20, 'C') 0.0
(21, 'W') 0.0
(22, 'E') 0.0
(23, 'L') 0.0
(24, 'Y') 0.0
(25, 'S') 0.0
(26, 'L') 0.0
(27, 'E') 0.0
(28, 'H') 0.0
(29, 'G') 0.0
(30, 'I') 0.0
(31, 'K') 0.0
(32, 'P') 0.0
(33, 'D') 0.0
(34, 'G') 0.0
(35, 'H') 0.0
(36, 'L') 0.0
(37, 'E') 0.0
(38, 'D') 0.0
(39, 'G') 0.0
(40, 'L') 0.0
(41, 'S') 0.0
(42, 'K') 0.0
(43, 'P') 0.0
(44, 'K') 0.0
(45, 'G') 0.0
(46, 'G') 0.0
(47, 'E') 0.0
(48, 'E') 0.0
(49, 'G') 0.0
(50, 'F') 0.0
(51, 'S') 0.0
(52, 'T') 0.0
(53, 'F') 0.0
(54, 'F') 0.0
(55, 'H') 0.0
(56, 'E') 0.0
(57, 'T') 0.0
(58, 'G') 0.0
(59, 'Y') 0.0
(60, 'G') 0.0
(61, 'K') 0.0
(62, 'F') 0.0
(63, 'V') 0.0
(64, 'P') 0.0
(65, 'R') 0.0
(66, 'A') 0.0
(67, 'I') 0.0
(68, 'Y') 0.0
(69, 'V') 0.0
(70, 'D') 0.0


In [12]:
# 3.4
def update_alignment_df_with_REL_values(uniprot_name, pdb_name, chain_id, alignment_df, alignment_dict, aa_rel_dict):
    
    def lookup_value(row, alignment_dict, aa_rel_dict):
        # Construct the key from the current row's 'Index' and 'UniProt_AA'
        key = (row['Index'], row['UniProt_AA'])
        # Get the corresponding PDB key from the alignment_dict using the constructed key
        pdb_key = alignment_dict.get(key)
        # Get the REL value from aa_rel_dict using the PDB key, if it exists
        rel_value = aa_rel_dict.get(pdb_key) if pdb_key else None
        
        return rel_value

    # Use the apply() function to apply the lookup_value function to each row of alignment_df
    # Pass additional arguments alignment_dict and aa_rel_dict using args parameter
    # The result is a new 'Value' column in alignment_df
    alignment_df['ABS'] = alignment_df.apply(lookup_value, axis=1, args=(alignment_dict, aa_rel_dict))
    # Replace NaN values with an empty string in the 'Value' column
    alignment_df['ABS'] = alignment_df['ABS'].fillna('')
    
    # Write the updated DataFrame to a CSV file
    filename = f'{pdb_name}_{chain_id}_{uniprot_name}_cal.csv'
    alignment_df.to_csv(filename, index=False)
    
    return alignment_df

alignment_df = update_alignment_df_with_REL_values(uniprot_name, pdb_name, chain_id, alignment_df, alignment_dict, aa_rel_dict)
print(f'\nExported result in file {pdb_name}_{chain_id}_{uniprot_name}_cal.csv ')
print(alignment_df)


Exported result in file 5w3f_A_P09733_cal.csv 
     Index  UniProt_Index UniProt_AA PDB_Index PDB_AA  ABS
0        1              1          M         1      M  0.0
1        2              2          R         2      R  0.0
2        3              3          E         3      E  0.0
3        4              4          V         4      V  0.0
4        5              5          I         5      I  0.0
..     ...            ...        ...       ...    ...  ...
442    443            443          E                -     
443    444            444          E                -     
444    445            445          E                -     
445    446            446          E                -     
446    447            447          F                -     

[447 rows x 6 columns]


In [13]:
# Example
# Calculate the moment(M) of proteins (The truncated protein in the elute)
def calculate_moment(r,n,m):
    # Initialize the sum
    total_sum = 0   
    # Calculate the sum of r * (i - m) from i=0 to n
    for i in range(n + 1):  # range(n+1) because the upper limit is inclusive
        total_sum += r * (i - m)
    return total_sum

In [14]:
# Part 4 Calculate the moment(M) of proteins (The truncated protein in the elute)
'''
- n is the total length of UniProt sequence (n = max(UniProt_Index)).
- m is the middle value that can separate the UniProt sequence into equivalent upper and lower two sections.
  This is to make sure the REl value for aa counting form both ends has the same weight
  (so they can counter each other).
- i is the UniProt_Index of each PDB aa.
- r is the REL value of each PDB aa, when the REL is not 0.0 and there is a corresponding UniProt aa for the PDB aa.
  This is to make sure the calculation is based on the UniProt sequence
  (So any addition from the PDB sequence would not affect the calculation).
'''

# The absolute moment
def calculate_absolute_moment(alignment_df):
    # Determine the middle position 'm' of the sequence
    alignment_df['UniPort_Index'] = pd.to_numeric(alignment_df['UniProt_Index'], errors='coerce')
    n = alignment_df['UniPort_Index'].max()
    print(f'n = {n}')
    m = (n / 2) + 0.5 
    print(f'm = {m}')

    # Initialize the sum
    total_sum = 0
    # Iterate through each row of the DataFrame
    for index, row in alignment_df.iterrows():
        # Check if 'ABS' is numeric, not blank, greater than 0.0, and 'UniProt_AA' is not '-'
        if pd.notnull(row['ABS']) and row['ABS'] != '' and row['ABS'] > 0.0 and row['UniProt_AA'] != '-':
            r = float(row['ABS'])
            i = row['UniPort_Index']
            sum = r * (i - m)
            # print(i)
            # print(f'sum = {r} * ({i} - {m}) = {sum}')
            
            total_sum += sum
    
    # Round the total sum to 2 decimal places after the loop
    total_sum = round(total_sum, 2)

    print(f'Absolute Moment(M_abs) = {total_sum}')
    return total_sum

print(f'For PDB model {pdb_name}, chain {chain_id} encoded protein, with respect to the UniProt sequence {uniprot_name}')
M_abs = calculate_absolute_moment(alignment_df)


For PDB model 5w3f, chain A encoded protein, with respect to the UniProt sequence P09733
n = 447
m = 224.0
Absolute Moment(M_abs) = -6813.76


In [15]:
# The relative moment
def calculate_relative_moment(alignment_df):
    # Convert 'UniProt_Index' to numeric and handle errors
    alignment_df['UniProt_Index'] = pd.to_numeric(alignment_df['UniProt_Index'], errors='coerce')
    n = alignment_df['UniProt_Index'].max()
    print(f'n = {n}')
    m = (n / 2) + 0.5 
    print(f'm = {m}')
    
    # Initialize values
    total_sum = 0
    # Handle non-numeric values in 'ABS' column
    alignment_df['ABS'] = pd.to_numeric(alignment_df['ABS'], errors='coerce')
    # Drop rows with NaN values in 'ABS' column
    alignment_df = alignment_df.dropna(subset=['ABS'])
    total_r = alignment_df['ABS'].sum()
    
    # Iterate through each row of the DataFrame
    for index, row in alignment_df.iterrows():
        # Check if 'ABS' is numeric, not blank, greater than 0.0, and 'UniProt_AA' is not '-'
        if pd.notnull(row['ABS']) and row['ABS'] != '' and row['ABS'] > 0.0 and row['UniProt_AA'] != '-':
            r = float(row['ABS'])
            i = row['UniProt_Index']
            # Calculate the relative position with respect to 'm' for for odd residue number
            if n % 2 == 1:
                if i == m:
                    relative_i = 0
                else:
                    relative_i = (i - m) / n
            # Calculate the relative position with respect to 'm' for for odd residue number
            if n % 2 == 0:
                if i < m:
                    relative_i = ((i - m) - 0.5) / n
                elif i > m:
                    relative_i = ((i - m) + 0.5) / n
            
            relative_r = r / total_r
            
            line_sum = relative_r * relative_i
            print(f'sum = {relative_r} * {relative_i} = {line_sum}')
            
            total_sum += line_sum
    
    # Round the total sum to 2 decimal places after the loop
    total_sum = round(total_sum, 2)

    print(f'Relative Moment(M_rel) = {total_sum}')
    return total_sum

print(f'For PDB model {pdb_name}, chain {chain_id} encoded protein, with respect to the UniPort sequence {uniprot_name}')
M_rel = calculate_relative_moment(alignment_df)


For PDB model 5w3f, chain A encoded protein, with respect to the UniPort sequence P09733
n = 447
m = 224.0
sum = 0.03534147546938362 * -0.47651006711409394 = -0.016840568847827093
sum = 0.0023342642601513696 * -0.46756152125279643 = -0.0010914121484824077
sum = 0.014088952141627909 * -0.3400447427293065 = -0.004790874106325374
sum = 0.004876944972101969 * -0.3378076062639821 = -0.001647469106906929
sum = 0.047667819904366616 * -0.33557046979865773 = -0.015995912719586114
sum = 0.001816200508536142 * -0.3333333333333333 = -0.0006054001695120473
sum = 0.01183209970643054 * -0.3288590604026846 = -0.0038910931920476273
sum = 0.005359280189123043 * -0.32662192393736017 = -0.0017504584062907477
sum = 0.0047221213221939696 * -0.319910514541387 = -0.0015106562619099275
sum = 0.04070671041427235 * -0.2841163310961969 = -0.01156544121389841
sum = 0.042713463107310654 * -0.28187919463087246 = -0.01204003658058421
sum = 0.01857883798895988 * -0.2796420581655481 = -0.0051954244935570135
sum = 0.037